In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import least_squares
import math
import pandas as pd
import csv
import datetime
from scipy import stats
from scipy import optimize
from datetime import date, timedelta
import random
import requests
from scipy.stats import truncnorm
import datetime

import statsmodels as sm
#import statsmodels.api as sm

from PIL import Image
from scipy.stats import norm

from statsmodels.tsa.vector_ar.var_model import VAR

import functools

/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Read data

## Cases

In [2]:
df = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/covid.csv')

## Population

In [3]:
#pop = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/populacao_municipio.csv')

## Mobility data

In [4]:
mob = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/distancing_series.csv')

## Stringency

In [5]:
strg = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/stringency_series.csv')

# Functions

In [6]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [7]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

# Create dataframe to analyse the following time series:

    * Daily number of cases
    * Acumulated number of cases
    * SMRI
    * Stringency
    * Rt

In [8]:
df = df[(df.date >= '2020-03-01') & (df.date <= '2020-05-22')]

In [9]:
mob['date2'] = pd.to_datetime(mob.date)

In [10]:
mob.head(2)

,state,name,indUF,indCAP,indInland,date,date2
0,12,AC,33.924855,34.104342,33.770914,01-Mar-2020 00:00:00,2020-03-01
1,12,AC,34.429550,34.609595,34.275130,02-Mar-2020 00:00:00,2020-03-02


In [11]:
mob = mob[(mob.date2 <= '2020-05-22')]

In [13]:
series_state = df.groupby(['state', 'date'])['new_confirmed'].sum().reset_index()

In [25]:
for value in mob.name.unique():
    # select the state
    s = value
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # test stationarity
    
    s1 = 'indUF'
    s2 = 'new_confirmed'
    
    # Differencing to get the data stationary
    
    casesDiff = cases.filter([s1,s2]).diff().dropna()
    
    casesDiff = casesDiff.apply(pd.to_numeric)
    
    print('\x1b[0;30;43m'+ 'stationarity of the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')
    
    cases.filter([s1,s2]).apply(functools.partial(test_stationarity))
    
    print('\x1b[0;30;43m'+ 'stationarity of the differentiated series {}'.format(s1) + ' and {}'.format(s2)+'\x1b[0m')
    
    casesDiff.apply(functools.partial(test_stationarity))
    

AC
stationarity of the series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -2.306101
p-value                         0.170008
#Lags Used                      7.000000
Number of Observations Used    59.000000
Critical Value (1%)            -3.546395
Critical Value (5%)            -2.911939
Critical Value (10%)           -2.593652
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  2.076605
p-value                         0.998765
#Lags Used                      8.000000
Number of Observations Used    58.000000
Critical Value (1%)            -3.548494
Critical Value (5%)            -2.912837
Critical Value (10%)           -2.594129
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                -6.449136e+00
p-value                        1.539055e-08
#Lags Used                     6.000000e+00
Number of Observations Used    5.900000e+01
Critica

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Test Statistic                 -3.270921
p-value                         0.016229
#Lags Used                      6.000000
Number of Observations Used    56.000000
Critical Value (1%)            -3.552928
Critical Value (5%)            -2.914731
Critical Value (10%)           -2.595137
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                 -4.964141
p-value                         0.000026
#Lags Used                      5.000000
Number of Observations Used    57.000000
Critical Value (1%)            -3.550670
Critical Value (5%)            -2.913766
Critical Value (10%)           -2.594624
dtype: float64
BA
stationarity of the series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -4.344613
p-value                         0.000371
#Lags Used                     10.000000
Number of Observations Used    67.000000
Critical Value (1%)            -3.531955
Critical Value (5%)            -2.905755
Critical Value (10%)           -2.

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

stationarity of the series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -0.963641
p-value                         0.766293
#Lags Used                      7.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  0.893169
p-value                         0.993001
#Lags Used                      6.000000
Number of Observations Used    65.000000
Critical Value (1%)            -3.535217
Critical Value (5%)            -2.907154
Critical Value (10%)           -2.591103
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -5.070563
p-value                         0.000016
#Lags Used                      6.000000
Number of Observations Used    64.000000
Critical Value (1%)   

...

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Test Statistic                 -4.605642
p-value                         0.000126
#Lags Used                     11.000000
Number of Observations Used    67.000000
Critical Value (1%)            -3.531955
Critical Value (5%)            -2.905755
Critical Value (10%)           -2.590357
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  4.457038
p-value                         1.000000
#Lags Used                     10.000000
Number of Observations Used    68.000000
Critical Value (1%)            -3.530399
Critical Value (5%)            -2.905087
Critical Value (10%)           -2.590001
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -2.201161
p-value                         0.205824
#Lags Used                      7.000000
Number of Observations Used    70.000000
Critical Value (1%)            -3.527426
Critical Value (5%)            -2.903811
Critical Value (10%)  

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Test Statistic                 -0.832206
p-value                         0.809478
#Lags Used                      7.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  0.599158
p-value                         0.987595
#Lags Used                      7.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                -5.662133e+00
p-value                        9.327633e-07
#Lags Used                     6.000000e+00
Number of Observations Used    6.400000e+01
Critical Value (1%)           -3.536928e+00
Critical Value (5%)           -2.907887e+00
Crit

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: RuntimeWarning: divide by zero encountered in log
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [26]:
for value in mob.name.unique():
    # select the state
    s = value
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    
    # Analyse the lag dependence using Var
    
    s1 = 'indUF'
    s2 = 'new_confirmed'
    
    # Differencing to get the data stationary
    
    casesDiff = cases.filter([s1,s2]).diff().dropna()
    
    casesDiff = casesDiff.apply(pd.to_numeric)
    
    print('\x1b[0;30;43m'+ 'Var of the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')

    model = VAR(cases.filter([s1,s2]))
    
    model1 = model.select_order()
    print(model1.summary())
    

AC
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        9.789       9.862   1.784e+04       9.817
1        7.568       7.785       1937.       7.653
2        7.295       7.657       1474.       7.435
3        7.132      7.639*       1255.       7.328
4        7.259       7.910       1428.       7.511
5        7.355       8.151       1581.       7.664
6        7.415       8.355       1690.       7.780
7        7.413       8.498       1703.       7.834
8        7.251       8.480       1466.       7.728
9       6.591*       7.966      770.7*      7.124*
10       6.636       8.155       822.8       7.225
11       6.722       8.386       920.6       7.367
--------------------------------------------------
AL
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         

<ipython-input-26-c5b82fc0e335>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-26-c5b82fc0e335>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-26-c5b82fc0e335>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        11.39       11.46   8.836e+04       11.42
1        8.332      8.553*       4156.       8.417
2        8.239       8.607       3790.      8.381*
3        8.328       8.844       4151.       8.527
4        8.188       8.851       3621.       8.444
5        8.259       9.069       3908.       8.572
6        8.129       9.087       3458.       8.498
7        8.149       9.254       3567.       8.576
8       7.989*       9.242      3082.*       8.472
9        8.052       9.451       3342.       8.591
10       8.139       9.686       3733.       8.735
11       8.121       9.815       3778.       8.775
--------------------------------------------------
GO
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
-------------------------------------

<ipython-input-26-c5b82fc0e335>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-26-c5b82fc0e335>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-26-c5b82fc0e335>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        10.02       10.09   2.255e+04       10.05
1        5.908      6.116*       368.0       5.989
2        5.835       6.181       342.3      5.971*
3        5.815       6.300      336.1*       6.005
4        5.828       6.451       341.1       6.072
5        5.840       6.601       346.4       6.138
6        5.903       6.802       370.9       6.255
7        5.863       6.901       359.2       6.270
8       5.802*       6.978       341.0       6.263
9        5.869       7.184       369.4       6.384
10       5.897       7.350       386.0       6.467
11       5.930       7.521       406.6       6.553
--------------------------------------------------
RJ
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQI

<ipython-input-26-c5b82fc0e335>:22: RuntimeWarning: divide by zero encountered in log
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
<ipython-input-26-c5b82fc0e335>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
<ipython-input-26-c5b82fc0e335>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [31]:
for value in mob.name.unique():
    # select the state
    s = value
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    
    # calculate correlation between series without lag and any previous analysis
    
    s1 = 'indUFLog'
    s2 = 'new_confirmedLog'
    
    print('\x1b[0;30;43m'+ 'Crosscorrelation between the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')

    
    r, p = stats.pearsonr(cases.dropna()[s1], cases.dropna()[s2])
    print(f"Scipy computed Pearson r: {r} and p-value: {p}")
    
    r, p = stats.spearmanr(cases.dropna()[s1], cases.dropna()[s2])
    print(f"Scipy computed Spearman r: {r} and p-value: {p}")
    
    r, p = stats.kendalltau(cases.dropna()[s1], cases.dropna()[s2])
    print(f"Scipy computed Kendall's tau: {r} and p-value: {p}")

AC
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: -0.2143467269238654 and p-value: 0.08154476324126608
Scipy computed Spearman r: -0.34937564445894287 and p-value: 0.0037571225003498673
Scipy computed Kendall's tau: -0.2615526243690664 and p-value: 0.002117024021865508
AL
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: 0.25167147277619767 and p-value: 0.028300938424832646
Scipy computed Spearman r: 0.014853729460754413 and p-value: 0.8986612441834195
Scipy computed Kendall's tau: 0.022247872417182707 and p-value: 0.7843109791221065
AM
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: 0.4456836099662927 and p-value: 9.820498383205705e-05
Scipy computed Spearman r: -0.07257753850998798 and p-value: 0.5475148125910658
Scipy computed Kendall's tau: -0.09418039738457037 and p-value: 0.2472603132579939
AP
Crosscorrelation between the series indUFLog and new_confi

<ipython-input-31-5032b3acccaa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-31-5032b3acccaa>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-31-5032b3acccaa>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda


MA
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: -0.0748462849007828 and p-value: 0.5566702905357901
Scipy computed Spearman r: -0.0280990233900752 and p-value: 0.8255555992364424
Scipy computed Kendall's tau: -0.023904572186687872 and p-value: 0.7808590525752981
MG
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: 0.3827709784218182 and p-value: 0.0006435445319646433
Scipy computed Spearman r: -0.12069687653045959 and p-value: 0.29900241115769216
Scipy computed Kendall's tau: -0.15939630227296 and p-value: 0.04293981891767204
MS
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: -0.32351372048282845 and p-value: 0.006297850832342396
Scipy computed Spearman r: -0.4039116867343673 and p-value: 0.000525290620089356
Scipy computed Kendall's tau: -0.3066117596202487 and p-value: 0.0002518334580260932
MT
Crosscorrelation between the series indUFLog and new_confir

<ipython-input-31-5032b3acccaa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-31-5032b3acccaa>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-31-5032b3acccaa>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Scipy computed Spearman r: 0.027726848039585868 and p-value: 0.8083461586589481
Scipy computed Kendall's tau: -0.1103342076254781 and p-value: 0.15410530050353594
RN
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: 0.17557504962218484 and p-value: 0.14015444455109738
Scipy computed Spearman r: -0.08000212890579624 and p-value: 0.5041132616536474
Scipy computed Kendall's tau: -0.07126784938460416 and p-value: 0.3895073048826255
RO
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: -0.7520124124032282 and p-value: 7.937404324299137e-13
Scipy computed Spearman r: -0.8099479432459623 and p-value: 5.305715012293492e-16
Scipy computed Kendall's tau: -0.6259572563462196 and p-value: 1.1322506734882407e-12
RR
Crosscorrelation between the series indUFLog and new_confirmedLog
Scipy computed Pearson r: -0.4649595614802802 and p-value: 0.00012340783761704326
Scipy computed Spearman r: -0.4265743538906569 and p-valu

<ipython-input-31-5032b3acccaa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-31-5032b3acccaa>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-31-5032b3acccaa>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [36]:
for value in mob.name.unique():
    # select the state
    s = value
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    
    # croscorrelation with diferent lags 
    
    s1 = 'indUFLog'
    s2 = 'new_confirmedLog'
    
    print('\x1b[0;30;43m'+ 'Crosscorrelation with lags between the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')
    
    xcov = [crosscorr(cases[s1], cases[s2], lag=i) for i in range(40)]
    print(xcov)
    
    
    #x= cases.indUF
    #y= cases.new_confirmedLog

    #n =len(x)
    #l1 = (-1/n) + (2/math.sqrt(n))

    #fig, [ax1,ax2] = plt.subplots(3, 1)
    #ax1.xcorr(x, y, usevlines=True, maxlags=None, normed=True, lw=2)

    #ax1.axhline(l1, color='blue', linestyle='dashed', lw=1)

    #ax1.grid(True)

    #ax2.acorr(x, usevlines=True, normed=True, maxlags=None, lw=2)
    #ax2.grid(True)
    
    #ax2.scatter(cases.new_confirmedLog,cases.indUF, marker='o')
    #ax2.legend('new_confirmedLog','indUF')
    
    #ax3.plot(xcov, marker='o')
    #ax3.axhline(l1, color='blue', linestyle='dashed', lw=1)
    #ax3.axhline(-l1, color='blue', linestyle='dashed', lw=1)
    #ax3.axis(ymin=-1,ymax=1)

    #plt.show()

AC
Crosscorrelation with lags between the series indUFLog and new_confirmedLog
[-0.2143467269238654, -0.3101818113619328, -0.39383173689471795, -0.4182352337311107, -0.43638120850869083, -0.44171807770184246, -0.428140751311934, -0.3778727645682514, -0.31945366246704393, -0.26368998416305683, -0.20898205481264365, -0.15316009207765724, -0.054164685374863845, 0.00926802171794559, 0.0769328202418705, 0.10134302641726727, 0.09950438558596071, 0.09829834049090724, 0.1436121633938653, 0.20077454127305083, 0.2576978497352194, 0.22577988007887717, 0.2847562878544216, 0.3259749698965456, 0.37472020234174047, 0.42830871146159744, 0.4882701196643113, 0.4576345582219054, 0.5331458626512281, 0.5534270351668172, 0.6002949964056978, 0.5959902796978916, 0.5869688893991782, 0.5238619127237081, 0.520753977911176, 0.5302251389890621, 0.48159411582022865, 0.3649730624424671, 0.391742046874847, 0.27329087234342814]
AL
Crosscorrelation with lags between the series indUFLog and new_confirmedLog
[0.251671472

<ipython-input-36-124361608a19>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-36-124361608a19>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-36-124361608a19>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[-0.10410027290790193, -0.15961855923356455, -0.2303403801640213, -0.30525751399163303, -0.39030772382940077, -0.4891075528980694, -0.5602520924431682, -0.6108896643716036, -0.6402085774033265, -0.6622138794640549, -0.6652029477332001, -0.6745207335170793, -0.6600717714867308, -0.7118924002896625, -0.7550379287805545, -0.7521341137582336, -0.7404699134656711, -0.8103639345370381, -0.813278944592094, -0.8101528948096216, -0.8115097327994567, -0.8279385136748506, -0.8123547885961591, -0.8060413323489737, -0.7848359732589926, -0.7683385905776112, -0.7773211705542145, -0.7622693113893424, -0.7686471308996282, -0.7762105119610357, -0.7708547482071545, -0.767365697918164, -0.7616087228318681, -0.7300197402395819, -0.7246348276468584, -0.6640818285739648, -0.6468107103294015, -0.5767681463140866, -0.5785044183334191, -0.5357307808208371]
MA
Crosscorrelation with lags between the series indUFLog and new_confirmedLog
[-0.07484628490078281, -0.18948422943515197, -0.25180858202857076, -0.26869457

[0.17557504962218495, 0.11366894906196051, 0.04367596621995958, -0.06219731215727629, -0.1505372640622369, -0.24480681610826716, -0.3756049125762738, -0.5081590559770143, -0.6093495615069491, -0.6645071926743961, -0.6314525303751255, -0.6289794739845423, -0.6397949009140581, -0.6249845156091905, -0.6491319602205845, -0.5848194790305047, -0.5447103983731871, -0.5391488192397464, -0.5633204413005398, -0.560567278528934, -0.5334506112139806, -0.44111986170247736, -0.43922311980023854, -0.3986432884152179, -0.4398824370695383, -0.42054577012955646, -0.36006655744235544, -0.2954846903518438, -0.30127966842887366, -0.24867290256367955, -0.28411227347371076, -0.14768985619052266, -0.11713224401867449, -0.12015178617705799, -0.20946232315267804, -0.2415173917391078, -0.3115045840315093, -0.25733371780251557, -0.19515042747697858, -0.09908724860021502]
RO
Crosscorrelation with lags between the series indUFLog and new_confirmedLog
[-0.7520124124032281, -0.7460224134255288, -0.7579053910722415, -

<ipython-input-36-124361608a19>:16: RuntimeWarning: divide by zero encountered in log
  cases['cumLog'] = np.log(cases.filter(['cumulative']))


# Analysis of the fit results

## Data results

In [ ]:
import sys
import glob

path = r'/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap'
filenames = glob.glob(path + "/*.csv")
filenames

In [ ]:
lst_df = []
for f in filenames:
    print(f)
    df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")     
    #df.columns= df.columns.str.lower()
    lst_df.append(df)

In [ ]:
df = pd.concat(lst_df,sort=True)
df.head()

## Calculation of the confidence intervals

In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.median(a), scipy.stats.sem(a)
    h = se * scipy.stats.skewnorm.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def day_date(t):
    t_date = []
    for days in t:
        days = np.nan_to_num(days)
        date = datetime.datetime(2020, 1, 1) + datetime.timedelta(int(days) - 1)
        t_date.append(date)

    return(t_date)

def r0(b,delta,gammaA=1/3.5,gammaS=1/4,p=0.2):

    R0a = b*delta/gammaA

    R0s = b/gammaS

    R0_geral = p*R0s + (1-p)*R0a
    
    return(R0_geral)

In [ ]:
dfs = []
for uf in df.state.unique():
    for area in df.type.unique():
    
        frame = df[(df.state == uf) & (df.type == area)]


        beta0= np.array(mean_confidence_interval(frame['beta0'], confidence=0.95))

        beta0Round = np.round(mean_confidence_interval(frame['beta0'], confidence=0.95), decimals=2)

        beta1 = np.round(mean_confidence_interval(frame['beta1'], confidence=0.95), decimals=2)

        beta2 = np.round(mean_confidence_interval(frame['beta2'], confidence=0.95), decimals=2)

        delta = np.array(mean_confidence_interval(frame['delta'], confidence=0.95))

        deltaRound = np.round(mean_confidence_interval(frame['delta'], confidence=0.95), decimals=2)

        tcut0 = np.round(mean_confidence_interval(frame['tcut0'], confidence=0.95), decimals=0)

        tcut1 = np.round(mean_confidence_interval(frame['tcut1'], confidence=0.95), decimals=0)

        tcut0_date = np.array(day_date(tcut0))

        tcut1_date = np.array(day_date(tcut1))

        R0 = np.round(r0(beta0,delta),2)
    
        data = np.array([beta0, beta1,delta,tcut0,tcut0_date,tcut1,tcut1_date,R0])
        df1 = pd.DataFrame(data=data, index=['beta0','beta1','delta','tcut0','tcut0_date','tcut1','tcut1_date','R0'],columns=['median','lower','upper']).transpose()
        df1['state'] = uf
        df1['type'] = area
        dfs.append(df1)

In [ ]:
dfs

In [ ]:
dfs_result = pd.concat(dfs,sort=True)

In [ ]:
#dfs_result.to_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/ic_par.csv')

# Comparing dates 

In [37]:
pars = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/ic_par.csv')

In [38]:
pars.head()

,Unnamed: 0,R0,beta0,beta1,delta,state,tcut0,tcut0_date,tcut1,tcut1_date,type
0,median,1.91,1.141240,1.08,0.312494,TO,128.0,2020-05-07,NaN,2019-12-31,estado
1,lower,1.71,1.071549,1.02,0.282560,TO,125.0,2020-05-04,NaN,2019-12-31,estado
2,upper,2.13,1.210931,1.13,0.342427,TO,131.0,2020-05-10,NaN,2019-12-31,estado
3,median,1.35,0.964620,0.88,0.213145,TO,129.0,2020-05-08,NaN,2019-12-31,capital
4,lower,1.16,0.882606,0.79,0.183743,TO,125.0,2020-05-04,NaN,2019-12-31,capital


In [51]:
pars.tcut0_date =  pd.to_datetime(pars.tcut0_date)

In [41]:
series_state.head()

,state,date,new_confirmed
0,AC,2020-03-17,3
1,AC,2020-03-18,0
2,AC,2020-03-19,1
3,AC,2020-03-20,3
4,AC,2020-03-21,4


In [53]:
series_state.date =  pd.to_datetime(series_state.date)

In [79]:
for value in series_state.state.unique():
    # select the state
    s = value

    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    p = pars[pars.state == s]
    
    mobs = mob[mob.name == s]
    mobs = mobs[(mobs[mobs.name == s].date2 >= cases.iloc[0,1])]
    
    mobs[mobs.indUF == max(mobs.indUF)]    
    
    d1 = cases.iloc[0,1]
    
    d2 = p.iloc[0,7]
    
    d3 = mobs.iloc[0,6]
    
    print(s, d1, d2, d3, 'tcut1 and first case ->', abs((d2 - d1).days), 'pick of mob and tcut1 ->', abs((d3 - d2).days))
    
    #mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]

AC 2020-03-17 00:00:00 2020-05-17 00:00:00 2020-03-17 00:00:00 tcut1 and first case -> 61 pick of mob and tcut1 -> 61
AL 2020-03-08 00:00:00 2020-04-28 00:00:00 2020-03-08 00:00:00 tcut1 and first case -> 51 pick of mob and tcut1 -> 51
AM 2020-03-13 00:00:00 2020-04-06 00:00:00 2020-03-13 00:00:00 tcut1 and first case -> 24 pick of mob and tcut1 -> 24
AP 2020-03-20 00:00:00 2020-04-09 00:00:00 2020-03-20 00:00:00 tcut1 and first case -> 20 pick of mob and tcut1 -> 20
BA 2020-03-06 00:00:00 2020-03-30 00:00:00 2020-03-06 00:00:00 tcut1 and first case -> 24 pick of mob and tcut1 -> 24
CE 2020-03-16 00:00:00 2020-04-02 00:00:00 2020-03-16 00:00:00 tcut1 and first case -> 17 pick of mob and tcut1 -> 17
DF 2020-03-07 00:00:00 2020-03-23 00:00:00 2020-03-07 00:00:00 tcut1 and first case -> 16 pick of mob and tcut1 -> 16
ES 2020-03-19 00:00:00 2020-04-14 00:00:00 2020-03-19 00:00:00 tcut1 and first case -> 26 pick of mob and tcut1 -> 26
GO 2020-03-12 00:00:00 2020-04-04 00:00:00 2020-03-12 00

In [78]:
mob

,state,name,indUF,indCAP,indInland,date,date2
0,12,AC,33.924855,34.104342,33.770914,01-Mar-2020 00:00:00,2020-03-01
1,12,AC,34.429550,34.609595,34.275130,02-Mar-2020 00:00:00,2020-03-02
2,12,AC,34.919644,35.118317,34.749249,03-Mar-2020 00:00:00,2020-03-03
3,12,AC,35.147736,35.333082,34.988770,04-Mar-2020 00:00:00,2020-03-04
4,12,AC,34.982379,35.166184,34.824735,05-Mar-2020 00:00:00,2020-03-05
...,...,...,...,...,...,...,...
2392,17,TO,40.665150,41.608438,40.454475,18-May-2020 00:00:00,2020-05-18
2393,17,TO,40.623109,41.506788,40.425748,19-May-2020 00:00:00,2020-05-19
2394,17,TO,40.477544,41.327409,40.287735,20-May-2020 00:00:00,2020-05-20
2395,17,TO,40.207128,41.051061,40.018643,21-May-2020 00:00:00,2020-05-21


In [66]:
s = 'SC'
mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
mobs[mobs.indUF == max(mobs.indUF)]

,state,name,indUF,indCAP,indInland,date,date2
2070,42,SC,63.448656,61.557285,63.619165,24-Mar-2020 00:00:00,2020-03-24


In [67]:
series_state[series_state.state == 'SC']

,state,date,new_confirmed
1605,SC,2020-03-12,2
1606,SC,2020-03-13,1
1607,SC,2020-03-14,2
1608,SC,2020-03-15,1
1609,SC,2020-03-16,1
...,...,...,...
1672,SC,2020-05-18,399
1673,SC,2020-05-19,238
1674,SC,2020-05-20,86
1675,SC,2020-05-21,111


In [65]:
pars[pars.state == 'SC']

,Unnamed: 0,R0,beta0,beta1,delta,state,tcut0,tcut0_date,tcut1,tcut1_date,type
180,median,3.99,1.798053,0.64,0.507495,SC,84.0,2020-03-24,NaN,2019-12-31,estado
181,lower,3.79,1.761289,0.61,0.481869,SC,83.0,2020-03-23,NaN,2019-12-31,estado
182,upper,4.21,1.834817,0.66,0.533120,SC,84.0,2020-03-24,NaN,2019-12-31,estado
183,median,1.43,0.789553,1.86,0.361720,SC,138.0,2020-05-17,NaN,2019-12-31,capital
184,lower,1.25,0.726145,1.76,0.330207,SC,134.0,2020-05-13,NaN,2019-12-31,capital
185,upper,1.62,0.852962,1.95,0.393234,SC,141.0,2020-05-20,NaN,2019-12-31,capital
186,median,3.78,1.785287,0.68,0.471016,SC,84.0,2020-03-24,NaN,2019-12-31,interior
187,lower,3.57,1.742275,0.65,0.445733,SC,83.0,2020-03-23,NaN,2019-12-31,interior
188,upper,4.00,1.828299,0.71,0.496299,SC,85.0,2020-03-25,NaN,2019-12-31,interior


# Analises diversas

In [ ]:
overall_pearson_r = new.corr().iloc[0,1]
print(f"Pandas computed Pearson r: {overall_pearson_r}")

# Compute rolling window synchrony
f,ax=plt.subplots(figsize=(7,3))
new.rolling(window=1,center=True).median().plot(ax=ax)
ax.set(xlabel='Time',ylabel='Pearson r')
ax.set(title=f"Overall Pearson r = {np.round(overall_pearson_r,2)}");

In [ ]:
# Set window size to compute moving window synchrony.
r_window_size = 8
# Interpolate missing data.
df_interpolated = new.interpolate()
# Compute rolling window synchrony
rolling_r = df_interpolated['new_confirmed'].rolling(window=r_window_size, center=True).corr(df_interpolated['isolated'])
f,ax=plt.subplots(2,1,figsize=(14,6),sharex=True)
new.rolling(window=10,center=True).median().plot(ax=ax[0])
ax[0].set(xlabel='Frame',ylabel='Smiling Evidence')
rolling_r.plot(ax=ax[1])
ax[1].set(xlabel='Frame',ylabel='Pearson r')
plt.suptitle("Smiling data and rolling window correlation")


In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
import functools
new.apply(functools.partial(test_stationarity))

In [ ]:
newLog.apply(functools.partial(test_stationarity))

In [ ]:
# Differencing to get the data stationary
newDiff = new.diff().dropna()
newDiff

newDiff.apply(functools.partial(test_stationarity))

newDiff = newDiff.apply(pd.to_numeric)

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(newDiff)

In [ ]:
# Best model order (automatic selection)
model1 = model.select_order()
model1.summary()

In [ ]:
#pip install --upgrade statsmodels

#!pip install "statsmodels==0.10.2"

In [ ]:
results = model.fit(maxlags=8, ic='aic')
results.summary()

In [ ]:
results.fittedvalues

In [ ]:
# Autocorrelation function (ACF) plot of the residuals with 2=pT bounds.
results.plot_acorr()
#fig.tight_layout()
#plt.savefig('ACF_residual.pdf')


In [ ]:
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
plot_acf(newLog.new_confirmed)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(new.new_confirmed, lags=50)

In [ ]:
AC, Acre; AL, Alagoas; AP, Amapá; AM, Amazonas; BA, Bahia; CE, Ceará; 

DF, Distrito Federal; ES, Espírito Santo; GO, Goiás; MA, Maranhão; MT, Mato Grosso; 

MS, Mato Grosso do Sul; MG, Minas Gerais; PA, Pará; PB, Paraíba; PR, Paraná; PE, Pernambuco;

PI, Piauí; RJ, Rio de Janeiro; RN, Rio Grande do Norte; RS, Rio Grande do Sul; 

RO, Rondônia; RR, Roraima; SC, Santa Catarina; SP, São Paulo; SE, Sergipe; TO, Tocantins.




print('AC',   3343*100000/8.944700e+05)
print('AL',     5300*100000/3.351092e+06)
print('AM',    27038*100000/4.207714e+06)
print('AP',    5655*100000/8.617730e+05)
print('BA',    12558*100000/1.493042e+07)
print('CE',    34573*100000/9.187886e+06)
print('DF',     5948*100000/3.052546e+06)
print('ES',     9520*100000/4.064052e+06)
print('GO',    2344*100000/7.116143e+06)
print('MA',    18767*100000/7.114598e+06)
print('MG',     5995*100000/2.129267e+07)
print('MS',      805*100000/2.809394e+06)
print('MT',     1271*100000/3.526220e+06)
print('PA',    22697*100000/8.690745e+06)
print('PB',     6882*100000/4.039277e+06)
print('PE',    25760*100000/9.617072e+06)
print('PI',     3258*100000/3.280697e+06)
print('PR',     2985*100000/1.151684e+07)
print('RJ',    33589*100000/1.736619e+07)
print('RN',     4598*100000/3.534165e+06)
print('RO',     2774*100000/1.796460e+06)
print('RR',     2296*100000/6.311810e+05)
print('RS',     5918*100000/1.142297e+07)
print('SC',     6458*100000/7.252502e+06)
print('SE',     4922*100000/2.319032e+06)
print('SP',    76871*100000/4.628933e+07)
print('TO',     2430*100000/1.590248e+06)           

In [ ]:
sig_uf = []
for value in mob.state_name:
    if value == 'Minas Gerais': value = 'MG'
    elif value == 'Ceará': value = 'CE'
    elif value == 'Pernambuco': value = 'PE'
    elif value == 'Mato Grosso do Sul': value = 'MS'
    elif value == 'Rio Grande do Sul': value = 'RS'
    elif value == 'Sergipe': value = 'SE'
    elif value == 'Rio Grande do Norte': value = 'RN'
    elif value == 'Acre': value = 'AC'
    elif value == 'Alagoas': value = 'AL'
    elif value == 'Bahia': value = 'BA'
    elif value == 'Rondônia': value = 'RO'
    elif value == 'Rio de Janeiro': value = 'RJ'
    elif value == 'Piauí': value = 'PI'
    elif value == 'Paraná': value = 'PR'
    elif value == 'Espírito Santo': value = 'ES'
    elif value == 'Pará': value = 'PA'
    elif value == 'Amapá': value = 'AP'
    elif value == 'Mato Grosso': value = 'MT'
    elif value == 'Paraíba': value = 'PB'
    elif value == 'Roraima': value = 'RR'
    elif value == 'Goiás': value = 'GO'
    elif value == 'São Paulo': value = 'SP'
    elif value == 'Distrito Federal': value = 'DF'
    elif value == 'Maranhão': value = 'MA'
    elif value == 'Santa Catarina': value = 'SC'
    elif value == 'Tocantins': value = 'TO'
    elif value == 'Amazonas': value = 'AM'
    sig_uf.extend([value])